# ParFuMor (version YAML & Objets)

Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display

In [2]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
annee="18"
numerosKalabas=[1,2,3,4,5]
numerosKalabas=[5]
nomKalabas=[repertoire+annee+"-K%d/"%num for num in numerosKalabas]

#nomKalabas=[home+"/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K2-Kalaba0304/"]

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

In [3]:
PFM.duplicateErrors=[]
for serie in nomKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/18-K5/
ADJ : Genre Nombre Cas
NOM : Genre Nombre Cas
DET : Nombre Genre
VER : Trans Temps Pers Genre
head stems
head stems,VER
head stems,VER,VI
head stems,VER,VD
head stems,VER,VT
head stems,ADJ
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2
head stems,NOM,N3
head stems,NOM,N4


In [4]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[u"enfANT"][0]
print PFM.lexique.formeLexeme[u"crocs"][0]

villageoise.N3.D
enfant.hyper.N2.C
croc.N1.B


In [5]:
paradigmes.getSigmas(["DET"])

[u'DET, Nombre=Sg, Genre=A',
 u'DET, Nombre=Sg, Genre=B',
 u'DET, Nombre=Sg, Genre=C',
 u'DET, Nombre=Sg, Genre=D',
 u'DET, Nombre=Du, Genre=A',
 u'DET, Nombre=Du, Genre=B',
 u'DET, Nombre=Du, Genre=C',
 u'DET, Nombre=Du, Genre=D',
 u'DET, Nombre=Pl, Genre=A',
 u'DET, Nombre=Pl, Genre=B',
 u'DET, Nombre=Pl, Genre=C',
 u'DET, Nombre=Pl, Genre=D']

In [6]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [7]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')

[('1A23u3i', 'CF=N3|N4, Nombre=Du')]

In [10]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
test
#test.index.sortlevel(0,ascending=False)

Nombre,Du,Pl,Sg
Genre,,,
A,ma-RAD-e,no-RAD-e,RAD-e-N
B,RAD-a-m,no-RAD-a,Nu-RAD-a
C,ma-RAD-e,RAD-e-n,Nu-RAD-e
D,ma-RAD-a,no-RAD-a,Nu-RAD-a


In [46]:
lignes=["Temps","Pers","Trans"]
colonne=["Genre"]
lignes=["Genre","Nombre"]
colonne=["Cas"]
for cf in ["N%d"%(i+1) for i in range(4)]:
    test=Lexeme("RAD",cf,"croc").getParadigm(lignes=lignes,colonne=colonne)
    test=test.reset_index().groupby(by=["Erg","Abs","Dat","Obl"]).agg(lambda x: ", ".join(set(x))).reset_index().set_index(lignes)
    print cf
    display(test)

lignes=["Cas","Genre"]
colonne=["Nombre"]
for cf in ["A%d"%(i+1) for i in range(2)]:
#for cf in ["DET"]:
    test=Lexeme("RAD",cf,"croc").getParadigm(lignes=lignes,colonne=colonne)
    test=test.reset_index().groupby(by=["Sg","Du","Pl"]).agg(lambda x: ", ".join(set(x))).reset_index().set_index(lignes)
#    test=test.reset_index().groupby(by=["Sg","Du","Pl"]).agg(lambda x: ", ".join(set(x))).reset_index().set_index(lignes)
    print cf
    display(test)

lignes=["Genre"]
colonne=["Nombre"]
for cf in ["DET"]:
    test=Lexeme("RAD",cf,"croc").getParadigm(lignes=lignes,colonne=colonne)
    test=test.reset_index().groupby(by=["Sg","Du","Pl"]).agg(lambda x: ", ".join(set(x))).reset_index().set_index(lignes)
    print cf
    display(test)

N1


Cas                                   Erg                     Abs  \
Genre      Nombre                                                   
A, C, B, D Du                    RAD-sa-s               ke-RAD-sa   
           Sg       racine(RAD)x1V23a3i-s  ke-racine(RAD)x1V23a3i   
           Pl      si-racine(RAD)x1V22a3o  ke-racine(RAD)x1V22a3o   

Cas                                   Dat                    Obl  
Genre      Nombre                                                 
A, C, B, D Du                   fo-RAD-sa               RAD-sa-p  
           Sg       racine(RAD)x1V23a3i-f  racine(RAD)x1V23a3i-p  
           Pl      ve-racine(RAD)x1V22a3o  racine(RAD)x1V22a3o-p

N2


Cas                                   Erg                     Abs  \
Genre      Nombre                                                   
A, C, B, D Sg                   ri-RAD-pi               RAD-pi-le   
           Du                   ri-RAD-sa               RAD-sa-le   
           Pl      ri-racine(RAD)x1V22a3o  racine(RAD)x1V22a3o-le   

Cas                                   Dat                     Obl  
Genre      Nombre                                                  
A, C, B, D Sg                   RAD-pi-vu               pu-RAD-pi  
           Du                   RAD-sa-vu               pu-RAD-sa  
           Pl      racine(RAD)x1V22a3o-vu  pu-racine(RAD)x1V22a3o

N3


...

,Nombre,Sg,Du,Pl
Cas,Genre,,,
Abs,B,RAD-te-Ze,racine(RAD)x1A22a3u-Ze,RAD-da-Ze
Dat,B,RAD-te-Ze-f,racine(RAD)x1A22a3u-Ze-f,RAD-da-Ze-f
Erg,B,RAD-te-Ze-p,racine(RAD)x1A22a3u-Ze-p,RAD-da-Ze-p
Obl,B,RAD-te-Ze-s,racine(RAD)x1A22a3u-Ze-s,RAD-da-Ze-s
Abs,D,RAD-te-fi,racine(RAD)x1A22a3u-fi,RAD-da-fi
Dat,D,RAD-te-fi-f,racine(RAD)x1A22a3u-fi-f,RAD-da-fi-f
Erg,D,RAD-te-fi-p,racine(RAD)x1A22a3u-fi-p,RAD-da-fi-p
Obl,D,RAD-te-fi-s,racine(RAD)x1A22a3u-fi-s,RAD-da-fi-s
Abs,A,ka-RAD-te,ka-racine(RAD)x1A22a3u,ka-RAD-da


A2


Nombre              Sg           Du                          Pl
Cas Genre                                                      
Abs C           RAD-te       RAD-be       racine(RAD)xki11V2u3e
    B        RAD-te-Ze    RAD-be-Ze    racine(RAD)xki11V2u3e-Ze
Dat B      RAD-te-Ze-f  RAD-be-Ze-f  racine(RAD)xki11V2u3e-Ze-f
Erg B      RAD-te-Ze-p  RAD-be-Ze-p  racine(RAD)xki11V2u3e-Ze-p
Obl B      RAD-te-Ze-s  RAD-be-Ze-s  racine(RAD)xki11V2u3e-Ze-s
Abs A        RAD-te-bu    RAD-be-bu    racine(RAD)xki11V2u3e-bu
Dat A      RAD-te-bu-f  RAD-be-bu-f  racine(RAD)xki11V2u3e-bu-f
Erg A      RAD-te-bu-p  RAD-be-bu-p  racine(RAD)xki11V2u3e-bu-p
Obl A      RAD-te-bu-s  RAD-be-bu-s  racine(RAD)xki11V2u3e-bu-s
Dat C         RAD-te-f     RAD-be-f     racine(RAD)xki11V2u3e-f
Erg C         RAD-te-p     RAD-be-p     racine(RAD)xki11V2u3e-p
Obl C         RAD-te-s     RAD-be-s     racine(RAD)xki11V2u3e-s
Abs D        me-RAD-te    me-RAD-be    me-racine(RAD)xki11V2u3e
Dat D      me-RAD-te-f  me-RAD-be-f  me-racine(RAD)xki11V2u3e-f
Erg D      me-RAD-te-p  me-RAD-be-p  me-racine(RAD)xki11V2u3e-p
Obl D      me-RAD-te-s  me-RAD-be-s  me-racine(RAD)xki11V2u3e-s

DET


Nombre,Sg,Du,Pl
Genre,,,
B,Nu-RAD-a,RAD-a-m,no-RAD-a
D,Nu-RAD-a,ma-RAD-a,no-RAD-a
C,Nu-RAD-e,ma-RAD-e,RAD-e-n
A,RAD-e-N,ma-RAD-e,no-RAD-e


In [39]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'A': {('Darig', u'louve'),
  ('Tibiw', u'for\xeat'),
  ('budug', u'soeur'),
  ('daNafi', u'balai'),
  ('dazaT', u'viande'),
  ('kaleb', u'Kaleb'),
  ('leZif', u'couteau'),
  ('mibaz', u'ombre'),
  ('navuwo', u'lune'),
  ('nikol', u'Nicole'),
  ('pegom', u'chambre'),
  ('roNimu', u'sort'),
  ('rupilu', u'chat'),
  ('tazas', u'caf\xe9'),
  ('veziN', u'homme'),
  ('wagop', u'enfant'),
  ('zaser', u'lit')},
 u'B': {('fusuk', u'plaine'),
  ('gezoS', u'blessure'),
  ('katiSa', u'Katisha'),
  ('kigudu', u'th\xe9'),
  ('meSem', u'nuit'),
  ('nabil', u'Nabil'),
  ('neNasu', u'combat'),
  ('pisow', u'arbre'),
  ('ruteTi', u'table'),
  ('sopib', u'coup'),
  ('tuSul', u'col\xe8re'),
  ('turaj', u'infirmi\xe8re'),
  ('viDib', u'main'),
  ('wiras', u'autruche'),
  ('wulena', u'croc')},
 u'C': {('Dudav', u'mort'),
  ('Duragi', u'loup'),
  ('Nizit', u'combattant'),
  ('Zetufa', u'corps'),
  ('Zuzuj', u'souris'),
  ('fibuN', u'\u0153uf'),
  ('gopefa', u'chasseur'),
  ('kofop', u'milieu'),
  ('posuso'

In [32]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

courageux.A2 tinig
entrer.VI.V1 wobek
infirmière.N1.B turaj
trois.A1 Nopot
fille.N2.D wugup
donner.VD.V1 gunit
offrir.VD.V2 keSeS
souffrance.N1.D paruz
planter.VT.V2 dezaf
poisson.N1.C sijid
souris.N4.C Zuzuj
rejoindre.VT.V2 ZiNaf
coyote.N4.C tamin
courir.VI.V1 gatiN
grand.A2 zubug
vers taje
village.N2.D Surubu
ENTRE lof
envahir.VT.V1 nujet
Nicole.N1.A nikol
effrayé.A1 rubaf
de zaS
sort.N2.A roNimu
main.N3.B viDib
lit.N3.A zaser
chef.N4.D pavuf
couteau.N4.A leZif
loup.N1.C Duragi
milieu.N2.C kofop
lueur.N2.C posuso
voir.VT.V2 mewuz
suivant.A1 Tumod
profond.A2 kikow
visage.N2.D Nanop
disparaître.VI.V1 Dazak
café.N3.A tazas
fuir.VI.V2 Sinez
acheter.VD.V1 riril
gros.A1 dugul
furieux.A2 sorawu
aller.VI.V2 wugar
blessé.A2 vepip
DEM b
chatte.N3.D diboZ
enfant.hyper.N2.C wigap
combattant.N4.C Nizit
DEF k
Nabil.N2.B nabil
œuf.N3.C fibuN
chambre.N2.A pegom
coup.N1.B sopib
cuisine.N4.D pubabu
recouvrir.VT.V2 witoj
sur vom
villageois.N3.C zalav
viande.N2.A dazaT
coussin.N4.D Nazet
jeter.VT.V1 poz

In [ ]:
temp=test.reset_index()
temp["Nombre"]=pd.Categorical(temp["Nombre"],categories=["Sg","Du","Pl"],ordered=True)
temp.sort_values("Nombre")